In [1]:
import os, pandas as pd
from pathlib import Path

base = Path("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data")
os.listdir(base)

master = pd.read_excel(base / "codige_with_cci_oncology_farmaco_predictors.xlsx")
adr = pd.read_excel(base / "codige_adr_clean_english.xlsx")

(master.shape, master.columns[:20]), (adr.shape, adr.columns)

C:\Users\HP\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\HP\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


(((412, 137),
  Index(['patient_id', 'birth_date', 'age', 'age_group', 'gender', 'ethnicity',
         'education_level', 'bmi_value', 'bmi_category', 'employment_status',
         'alcohol_consumption', 'smoking_status_binary', 'smoking_status_detail',
         'smoking_years', 'observation_start_date', 'observation_end_date',
         'observation_end_reason', 'tumor_diagnosis_date',
         'Oncology Unit Intake Date', 'tumor_type'],
        dtype='object')),
 ((2020, 11),
  Index(['patient_id', 'adr_description', 'adr_onset_date',
         'hospitalization_type', 'adr_ctcae_grade', 'adr_outcome',
         'adr_chemo_correlation', 'adr_chemo_action', 'adr_source_project',
         'adr_description_clean', 'adr_macro_category'],
        dtype='object')))

In [11]:
# Quick look at ADR data
adr.head(), adr["adr_ctcae_grade"].value_counts(dropna=False).head(10)

(     patient_id            adr_description adr_onset_date  \
 0  1_AO San Pio  adr tipica di panitumumab     2022-07-27   
 1  1_AO San Pio                     Anemia            NaT   
 2  1_AO San Pio                     Anemia            NaT   
 3  1_AO San Pio                     Anemia     2022-08-08   
 4  1_AO San Pio                     Anemia     2022-09-07   
 
                 hospitalization_type  adr_ctcae_grade  \
 0                           Ordinary              2.0   
 1                                NaN              1.0   
 2  Day Hospital / Day Service (PACC)              2.0   
 3                           Ordinary              2.0   
 4                           Ordinary              1.0   
 
                       adr_outcome  adr_chemo_correlation adr_chemo_action  \
 0                     Improvement                    2.0          Nessuna   
 1  Unchanged or Worsened Reaction                    3.0          Nessuna   
 2  Unchanged or Worsened Reaction        

In [12]:
# Standardise id columns and relevant ADR columns
adr = adr.rename(columns={
    "id_paziente": "patient_id",
    "insorgenza_dt": "adr_onset_date",
    "grado": "adr_ctcae_grade",
    "macrocategoria": "adr_macro_category"
})

# Ensure types
master["patient_id"] = master["patient_id"].astype(str).str.strip()
master["observation_start_date"] = pd.to_datetime(master["observation_start_date"], errors="coerce")
master["observation_end_date"] = pd.to_datetime(master["observation_end_date"], errors="coerce")

adr["patient_id"] = adr["patient_id"].astype(str).str.strip()
adr["adr_onset_date"] = pd.to_datetime(adr["adr_onset_date"], errors="coerce")
adr["adr_ctcae_grade"] = pd.to_numeric(adr["adr_ctcae_grade"], errors="coerce")

# Merge ADR with observation window
adr_m = adr.merge(
    master[["patient_id", "observation_start_date", "observation_end_date"]],
    on="patient_id",
    how="left"
)

censor_date = pd.to_datetime("2024-12-31")

# Define severe ADR at record level
adr_m["is_severe"] = adr_m["adr_ctcae_grade"] >= 3

# Within observation window and censoring
adr_m["within_window"] = (
    adr_m["is_severe"]
    & adr_m["adr_onset_date"].notna()
    & adr_m["observation_start_date"].notna()
    & adr_m["observation_end_date"].notna()
    & (adr_m["adr_onset_date"] >= adr_m["observation_start_date"])
    & (adr_m["adr_onset_date"] <= adr_m["observation_end_date"])
    & (adr_m["adr_onset_date"] <= censor_date)
)

# After observation end (severe)
adr_m["severe_adr_after_observation_end"] = (
    adr_m["is_severe"]
    & adr_m["adr_onset_date"].notna()
    & adr_m["observation_end_date"].notna()
    & (adr_m["adr_onset_date"] > adr_m["observation_end_date"])
)

# After censor (severe)
adr_m["severe_adr_after_censor"] = (
    adr_m["is_severe"]
    & adr_m["adr_onset_date"].notna()
    & (adr_m["adr_onset_date"] > censor_date)
)

# Observation window sanity check per record
adr_m["obs_end_before_start"] = (
    adr_m["observation_end_date"] < adr_m["observation_start_date"]
)

adr_m[["is_severe", "within_window"]].value_counts()

is_severe  within_window
False      False            1949
True       True               98
           False              27
Name: count, dtype: int64

In [13]:
# Aggregate to patient-level severe ADR outcome

def first_valid_date(dates):
    if dates.isna().all():
        return pd.NaT
    return dates.min()

grouped = adr_m.groupby("patient_id")

event_flag_bool = grouped["within_window"].any()
event_flag = event_flag_bool.map({True: "Yes", False: "No"})
event_date = grouped.apply(
    lambda g: first_valid_date(g.loc[g["within_window"], "adr_onset_date"])
)

any_after_obs_end = grouped["severe_adr_after_observation_end"].any()
any_after_censor = grouped["severe_adr_after_censor"].any()
any_obs_end_before_start = grouped["obs_end_before_start"].any()

# Build verification table
verif = grouped[["observation_start_date", "observation_end_date"]].first().reset_index()

verif["first_severe_adr_date"] = event_date.values
verif["severe_adr_flag"] = event_flag.values
verif["severe_adr_after_observation_end"] = any_after_obs_end.values
verif["severe_adr_after_censor"] = any_after_censor.values
verif["obs_end_before_start"] = any_obs_end_before_start.values

# Optionally add basic demographics for context
if "age_group" in master.columns and "bmi_value" in master.columns:
    demo = master[["patient_id", "age", "age_group", "bmi_value"]]
    verif = verif.merge(demo, on="patient_id", how="left")

verif.shape, verif["severe_adr_flag"].value_counts()


C:\Users\HP\AppData\Local\Temp\ipykernel_23788\164712831.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  event_date = grouped.apply(


((294, 11),
 severe_adr_flag
 No     240
 Yes     54
 Name: count, dtype: int64)

In [14]:
# Save the ADR severe outcome verification table
out_path = base / "severe_adr_outcome_verification_table_v3.xlsx"
verif.to_excel(out_path, index=False)

out_path

C:\Users\HP\AppData\Local\Temp\ipykernel_23788\2178940209.py:3: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  verif.to_excel(out_path, index=False)


WindowsPath('C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/severe_adr_outcome_verification_table_v3.xlsx')

In [1]:
import pandas as pd
from pathlib import Path

base = Path("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data")

# --- Load datasets ---
master = pd.read_excel(base / "codige_with_cci_oncology_farmaco_predictors.xlsx")
hosp = pd.read_excel(base / "codige_ricoveri_english.xlsx")

# --- Clean patient IDs ---
def clean_id(x):
    if pd.isna(x):
        return None
    return str(x).strip().replace("\u200b", "").replace("\xa0", " ")

master["patient_id_clean"] = master["patient_id"].apply(clean_id)
hosp["patient_id_clean"] = hosp["patient_id"].apply(clean_id)

# --- Convert dates to datetime ---
master["observation_start_date"] = pd.to_datetime(master["observation_start_date"], errors="coerce")
master["observation_end_date"] = pd.to_datetime(master["observation_end_date"], errors="coerce")
hosp["hospital_admission_date"] = pd.to_datetime(hosp["hospital_admission_date"], errors="coerce")
hosp["hospital_discharge_date"] = pd.to_datetime(hosp["hospital_discharge_date"], errors="coerce")

# --- Merge to align observation windows with each hospitalization ---
hosp_valid = hosp.merge(
    master[["patient_id_clean", "observation_start_date", "observation_end_date"]],
    on="patient_id_clean",
    how="left"
)

# --- Keep only valid hospitalizations within the observation window ---
hosp_valid_window = hosp_valid[
    (hosp_valid["hospital_admission_date"] >= hosp_valid["observation_start_date"]) &
    (hosp_valid["hospital_admission_date"] <= hosp_valid["observation_end_date"])
].copy()

# --- Count valid hospitalizations ---
hosp_count = (
    hosp_valid_window.groupby("patient_id_clean")["hospital_admission_date"]
    .count()
    .reset_index()
    .rename(columns={"hospital_admission_date": "valid_hospitalization_count"})
)

# --- Merge counts back into master ---
master = master.merge(hosp_count, on="patient_id_clean", how="left")

# Fill missing with 0
master["valid_hospitalization_count"] = master["valid_hospitalization_count"].fillna(0).astype(int)

# --- Create hospitalization flag (Yes if >=2 valid hospitalizations) ---
master["hospitalization_flag"] = master["valid_hospitalization_count"].apply(
    lambda x: "Yes" if x >= 1 else "No"
)

# --- Build the hospitalization verification table ---
verification_cols = [
    "patient_id",
    "observation_start_date",
    "observation_end_date",
    "valid_hospitalization_count",
    "hospitalization_flag",
]

# Add first two hospitalization dates for clarity (optional)
first_two = (
    hosp_valid_window.sort_values(["patient_id_clean", "hospital_admission_date"])
    .groupby("patient_id_clean")["hospital_admission_date"]
    .apply(lambda x: list(x)[:2])
    .reset_index()
    .rename(columns={"hospital_admission_date": "first_two_hospitalizations"})
)

# Merge with master
verification = (
    master.merge(first_two, on="patient_id_clean", how="left")
    [verification_cols + ["first_two_hospitalizations"]]
)

# --- Export verification table ---
output_path = base / "hospitalization_verification_table.xlsx"
verification.to_excel(output_path, index=False)

output_path


C:\Users\HP\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\HP\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\HP\AppData\Local\Temp\ipykernel_2508\4039864560.py:83: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  verification.to_excel(output_path, index=False)


WindowsPath('C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/hospitalization_verification_table.xlsx')

In [17]:
master.columns.tolist()

['patient_id',
 'birth_date',
 'age',
 'age_group',
 'gender',
 'ethnicity',
 'education_level',
 'bmi_value',
 'bmi_category',
 'employment_status',
 'alcohol_consumption',
 'smoking_status_binary',
 'smoking_status_detail',
 'smoking_years',
 'observation_start_date',
 'observation_end_date',
 'observation_end_reason',
 'tumor_diagnosis_date',
 'Oncology Unit Intake Date',
 'tumor_type',
 'breast_cancer_subtype',
 'colon_cancer_location',
 'stadio_TNM',
 'tumor_stage_roman',
 'histological_grade',
 'molecular_alterations',
 'mutations_present',
 'genotipo_DPYD',
 'genotipo_DPYD_type',
 'surgical_intervention',
 'surgery_date',
 'surgery_type',
 'surgery_type_specify',
 'prior_surgery',
 'prior_radiotherapy',
 'Number of Previous Treatment Lines',
 'Other Surgical Intervention',
 'surgery_complications',
 'reoperation_for_complication',
 'hospitalization_for_surgery_complication',
 'oncology_treatment_lines_n',
 'radiotherapy_status',
 'radiotherapy_start_date',
 'radiotherapy_end_dat

In [18]:
# Build death outcome verification table
cols = [
    "patient_id",
    "observation_start_date",
    "observation_end_date",
    "death_date",
    "death_outcome",
    "observation_end_reason",
    "survival_days",
]

death_verif = master[cols].copy()

out_path = "C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/death_outcome_verification_table_final.xlsx"
death_verif.to_excel(out_path, index=False)

out_path


C:\Users\HP\AppData\Local\Temp\ipykernel_23788\928774315.py:15: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  death_verif.to_excel(out_path, index=False)


'C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/death_outcome_verification_table_final.xlsx'

In [11]:
import pandas as pd


df = pd.read_excel("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/death_outcome_verification_table_final.xlsx")

dup_ids = [
    "16_AORN San Giuseppe Moscati",
    "25_AOU San Giovanni di Dio Ruggi di Aragona",
    "2_AORN San Giuseppe Moscati",
]

for pid in dup_ids:
    mask = df["patient_id"] == pid
    idx = df.index[mask]

    if len(idx) > 1:
        drop_idx = idx[2:]
        df = df.drop(drop_idx)

df = df.reset_index(drop=True)

print("Final death outcome shape:", df.shape)

df.to_excel("death_outcome_verification_table_final_v2.xlsx", index=False)


Final death outcome shape: (406, 7)


C:\Users\HP\AppData\Local\Temp\ipykernel_24692\1239186718.py:24: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel("death_outcome_verification_table_final_v2.xlsx", index=False)


In [2]:
import pandas as pd

df = pd.read_excel("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/hospitalization_verification_table.xlsx")

dup_ids = [
    "16_AORN San Giuseppe Moscati",
    "25_AOU San Giovanni di Dio Ruggi di Aragona",
    "2_AORN San Giuseppe Moscati",
]

for pid in dup_ids:
    mask = df["patient_id"] == pid
    idx = df.index[mask]

    if len(idx) > 1:
        drop_idx = idx[2:]
        df = df.drop(drop_idx)

df = df.reset_index(drop=True)

print("Final hospitalization shape:", df.shape)

df.to_excel("hospitalization_verification_table_v2.xlsx", index=False)


Final hospitalization shape: (406, 6)


C:\Users\HP\AppData\Local\Temp\ipykernel_2508\835309505.py:23: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel("hospitalization_verification_table_v2.xlsx", index=False)


In [9]:
import pandas as pd

df = pd.read_excel("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/severe_adr_outcome_verification_table_v3.xlsx")

dup_ids = [
    "16_AORN San Giuseppe Moscati",
    "2_AORN San Giuseppe Moscati",
]

for pid in dup_ids:
    mask = df["patient_id"] == pid
    idx = df.index[mask]

    if len(idx) > 1:
        drop_idx = idx[2:]
        df = df.drop(drop_idx)

df = df.reset_index(drop=True)

print("Final severe ADR shape:", df.shape)

df.to_excel("severe_adr_outcome_verification_table_v4.xlsx", index=False)


Final severe ADR shape: (290, 11)


C:\Users\HP\AppData\Local\Temp\ipykernel_24692\1567533923.py:22: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df.to_excel("severe_adr_outcome_verification_table_v4.xlsx", index=False)


In [3]:
# 1) Check max rows per patient_id in each table
for fname in [
    "codige_master_clean_predictors_only_v2.xlsx",
    "death_outcome_verification_table_final_v2.xlsx",
    "hospitalization_verification_table_v2.xlsx",
    "severe_adr_outcome_verification_table_v4.xlsx",
]:
    df = pd.read_excel(fname)
    print("===", fname)
    print("rows:", df.shape[0])
    print("max records per patient_id:",
          df["patient_id"].value_counts().max())
    print()


=== codige_master_clean_predictors_only_v2.xlsx
rows: 406
max records per patient_id: 2

=== death_outcome_verification_table_final_v2.xlsx
rows: 406
max records per patient_id: 2

=== hospitalization_verification_table_v2.xlsx
rows: 406
max records per patient_id: 2

=== severe_adr_outcome_verification_table_v4.xlsx
rows: 290
max records per patient_id: 2



In [5]:
import pandas as pd

# Load your files
verif = pd.read_excel("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/severe_adr_outcome_verification_table_v4.xlsx")
model = pd.read_excel("C:/Users/HP/OneDrive/Desktop/VERO_code/Phase_1/new_data/adr/severe_adr_model_matrix_v1.xlsx")

# Standardize patient_id formatting
def clean_id(x):
    if isinstance(x, str):
        x = x.strip()
        x = x.replace("ospedale del mare", "Ospedale del Mare")
        x = x.replace("Ospedale Del Mare", "Ospedale del Mare")
        x = x.replace("ospedale Del Mare", "Ospedale del Mare")
        x = x.replace("ospedale Del mare", "Ospedale del Mare")
        x = x.replace("ospedale del Mare", "Ospedale del Mare")
    return x

verif['patient_id'] = verif['patient_id'].apply(clean_id)
model['patient_id'] = model['patient_id'].apply(clean_id)

# Severe ADR flagged Yes: ground truth set = 52
verif_yes = set(verif.loc[verif['severe_adr_flag'] == "Yes", 'patient_id'])

# Severe ADR flagged Yes in model matrix = 51
model_yes = set(model.loc[model['severe_adr_flag'] == "Yes", 'patient_id'])

# Identify mismatch
missing_patient = verif_yes - model_yes

print("Patients with Yes in verification:", len(verif_yes))
print("Patients with Yes in model:", len(model_yes))
print("Missing patient(s):", missing_patient)

# Display the row(s) for review
missing_details = verif[verif['patient_id'].isin(missing_patient)]
print("\nDetails of missing patient:")
print(missing_details)


Patients with Yes in verification: 51
Patients with Yes in model: 51
Missing patient(s): set()

Details of missing patient:
Empty DataFrame
Columns: [patient_id, observation_start_date, observation_end_date, first_severe_adr_date, severe_adr_flag, severe_adr_after_observation_end, severe_adr_after_censor, obs_end_before_start, age, age_group, bmi_value]
Index: []
